In [2]:
# CODE ADAPTED FROM HW3

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Load the data**

In [3]:
train = pd.read_csv("../input/predict-volcanic-eruptions-ingv-oe/train.csv")
train.head()

,segment_id,time_to_eruption
0,1136037770,12262005
1,1969647810,32739612
2,1895879680,14965999
3,2068207140,26469720
4,192955606,31072429


In [4]:
train_dir = "../input/predict-volcanic-eruptions-ingv-oe/train/"
test_dir = "../input/predict-volcanic-eruptions-ingv-oe/test/"

In [5]:
def read_csv(index):
    train1 = pd.read_csv(train_dir + str(train.segment_id.iloc[index]) + ".csv")
    train1['timetoerupt'] = train.time_to_eruption.iloc[index]
    for feat in train1.drop('timetoerupt',1).columns:
        train1[feat] = train1[feat].mean()
    train1 = train1.sample(1)
    return (train1)

In [ ]:
# add all the training data sets into one dataframe
data = pd.DataFrame()
for idx in range(train.shape[0]):
    df = read_csv(idx)
    data=pd.concat([df,data])

In [ ]:
# replace null values with the mean value
for feat in data:
    data[feat] = data[feat].replace(np.nan, data[feat].mean())

In [ ]:
# print training data
print(data.shape)
data.head()

In [ ]:
# prepare submission document
sub = pd.read_csv("../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv")
sub.head()

In [ ]:
def read_csv(index):
    test1 = pd.read_csv(test_dir + str(sub.segment_id.iloc[index]) + ".csv")
    for feat in test1.columns:
        test1[feat] = test1[feat].mean()
    test1 = test1.sample(1)    
    return (test1)

In [ ]:
test = pd.DataFrame()
for idx in range(sub.shape[0]):
    df = read_csv(idx)
    test = pd.concat([df,test])

In [ ]:
# replace null values with mean, same as the training data
for feat in test:
    test[feat] = test[feat].replace(np.nan, test[feat].mean())

In [ ]:
# print test data
print(test.shape)
test.head()

In [ ]:
all_features = pd.concat((data.iloc[:, 1:-1], test.iloc[:, 1:]))
all_features.head()

In [ ]:
n_train = data.shape[0]
train_features = all_features[:n_train].values.astype(float)
test_features = all_features[n_train:].values.astype(float)
train_labels = data.timetoerupt.values.astype(float).reshape((-1, 1))

In [ ]:
from torch.utils.data import Dataset, DataLoader

class VolcanicEruptionDataset(Dataset):
    def __init__(self, train_features, train_labels):
        super().__init__()
        self.train_features = train_features
        self.train_labels = train_labels

    def __len__(self):
        return len(self.train_features)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return (self.train_features[idx], self.train_labels[idx])

# **Linear NN Model**

In [ ]:
import torch
from torch import nn

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=2, output_dim=1):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        y_pred = self.linear1(x)
        y_pred = self.relu(y_pred) # ReLU activation
        y_pred = self.linear2(y_pred)
        return y_pred

In [ ]:
class Log_RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

        
    def forward(self, yhat, y):
        ## Note: To further stabilize the value when the logarithm is taken, set the value of yhat less than 1 as 1.
        clipped_preds = torch.clamp(yhat, 1, float('inf'))
        return torch.sqrt(self.mse(torch.log(clipped_preds), torch.log(y)))


criterion = Log_RMSELoss()

In [ ]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, batch_size):
    train_ls, test_ls = [], []
    train_dataset = VolcanicEruptionDataset(train_features, train_labels)
    train_iter = DataLoader(train_dataset, batch_size, shuffle=True)
    # The Adam optimization algorithm is used here.
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    #criterion = nn.MSELoss()
    criterion = Log_RMSELoss()
    
    for epoch in range(num_epochs):
        for X, y in train_iter:
            yhat = net(X.float())
            loss = criterion(yhat, y.float())

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_ls.append(loss)
        if test_labels is not None:
            test_ls.append(loss)
    return train_ls, test_ls

In [ ]:
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = np.concatenate((X_train, X_part), axis=0)
            y_train = np.concatenate((y_train, y_part), axis=0)
    return X_train, y_train, X_valid, y_valid

In [ ]:
## util function
from matplotlib import pyplot as plt
from IPython import display

def semilogy(x_vals, y_vals, x_label, y_label, x2_vals=None, y2_vals=None,
             legend=None, figsize=(3.5, 2.5)):
    """Plot x and log(y)."""

    def set_figsize(figsize=(3.5, 2.5)):
        """Set matplotlib figure size."""
        display.set_matplotlib_formats('svg')
        plt.rcParams['figure.figsize'] = figsize

    set_figsize(figsize)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.semilogy(x_vals, y_vals)
    if x2_vals and y2_vals:
        plt.semilogy(x2_vals, y2_vals, linestyle=':')
        plt.legend(legend)
    plt.show()

In [ ]:
def k_fold(k, X_train, y_train, num_epochs,
           learning_rate, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = MLP(all_features.shape[1])
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,
                                   batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        if i == 0:
            semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'mse',
                        range(1, num_epochs + 1), valid_ls,
                        ['train', 'valid'])
        print('fold %d, train mse: %f, valid mse: %f' % (
            i, train_ls[-1], valid_ls[-1]))
    return train_l_sum / k, valid_l_sum / k

In [ ]:
k, num_epochs, lr, batch_size = 5, 64, 0.001, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr, batch_size)
print('%d-fold validation: avg train rmse: %f, avg valid rmse: %f'
      % (k, train_l, valid_l))

In [ ]:
def train_and_pred(train_features, test_feature, train_labels, test_data,
                   num_epochs, lr, batch_size):
    net = MLP(all_features.shape[1])
    train_ls, _ = train(net, train_features, train_labels, None, None,
                        num_epochs, lr, batch_size)
    semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'rmse')
    print('train rmse %f' % train_ls[-1])
    # apply the network to the test set
    preds = net(torch.from_numpy(test_features).float()).data.numpy()
    # reformat it for export to Kaggle
    sub['time_to_eruption'] = preds
    sub.to_csv('submission.csv', index=False)

In [ ]:
train_and_pred(train_features, test_features, train_labels, data,
               num_epochs, lr, batch_size)